# Script for **Image Acquisition**  (*raw* and *cropped*)

### i) Packages

In [1]:
import numpy as np
import cv2 as cv
from cv2 import VideoCapture as cap
import matplotlib.pyplot as plt
from skimage import io
import os
import math
import time

### ii) Define functions

In [2]:
def write_imgs(from_directory, to_directory, init_number = 1,
              classifier = 'haarcascade_frontalface_default.xml', scale_factor = 1.1, minNeighbors = 5,
              size = (224, 224), color = "RGB"):
    '''
    This function takes in input images and after face detection apply a crop and a resize to that images.
    
    @params:
    from_directory = where to find the photos
    to_directory = where to save the photos
    init_number = first number to start naming photos
    classifier = face detector pre-trained (XML file)
    scale_factor = value for classifier (from (1,inf), low values increase false positives)
    minNeighbors = value for classifier (high values decrease false positives)
    size = tuple, format of the saved image
    color = RGB or BW (to save)
    
    '''
    # viene assegnato il classificatore
    face_cascade = cv.CascadeClassifier(classifier)
    
    # variabili per conteggio ed errori
    flag = init_number
    error = 0
    
    for filename in os.listdir(from_directory):
        # check per leggere solamente file jpg o jpeg
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            # lettura dell'immagine            
            img = cv.imread(os.path.join(from_directory, filename))
            # conversione immagine necessaria per il face detector
            gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
            # esecuzione face detector
            faces = face_cascade.detectMultiScale(gray, scale_factor, minNeighbors)
            # check necessario per assegnare solo immagini in cui vengono trovati volti (altrimenti doppioni a catena!)
            if len(faces) == 1:
                for (x, y, w, h) in faces:
                    # ritaglio dell'immagine rispetto al "quadrato" trovato
                    roi_color = img[y:y+h, x:x+w]
                    # resize dell'immagine
                    n_img = cv.resize(roi_color, size)
                    
                    # salvataggio immagine
                    if color == "RGB":
                        cv.imwrite(os.path.join(to_directory, str(flag))+"_rgb.jpg", n_img)
                        flag += 1
                    elif color == "BW":
                        cv.imwrite(os.path.join(to_directory, str(flag))+"_bw.jpg", cv.cvtColor(n_img, cv.COLOR_RGB2GRAY))
                        flag += 1
                        
            # se non trova volti, prosegue qui            
            else:
                error += 1
                try:
                    # delete immagine per non creare doppioni (more safe!)
                    del faces, n_img
                # necessaria se non ha trovato volti (evitare crash code)
                except:
                    pass
                
    # output finale           
    print('Created {}/{} images cropped'.format(flag-1, flag+error-1))
                    
    

In [3]:
def acquisition_cam(directory, subject, init_number, counter_limit,
                    resolution = (1280, 720), fps = 30, time_sleep = 2, numbered = False):
    '''
    This function allow to access to pc webcam in order to acquire images of the subject.
    
    @params:
    directory = where to save the captured images
    subject = name of the person who is in photo
    init_number = first number to start naming photos
    counter_limit = if desired, limit the number of photos captured
    resolution = (width, height)
    fps = Frame Per Second
    time_sleep = sleep between two photos (second)
    numbered = Bool, necessary in order to apply counter_limit
    '''
    # assegnazione cam (0, prima della lista)
    cam = cv.VideoCapture(0)
    
    # setup configuration cam (resolution, fps)
    cam.set(cv.CAP_PROP_FRAME_WIDTH, resolution[0])
    cam.set(cv.CAP_PROP_FRAME_HEIGHT, resolution[1])
    cam.set(cv.CAP_PROP_FPS, fps)
    # rename windows
    cv.namedWindow("acquisition window")
    
    # variabili per conteggi
    img_counter = init_number
    counter = 1
    
    # ciclo acquisizione frameByframe
    while True:
        # lettura frame e assegnazione alla variabile
        ret, frame = cam.read()
        # riproduco frame per averlo in live (sulla finestra aperta)
        cv.imshow("acquisition window", frame)
        
        # se non va, esco dal ciclo senza rompere code
        if not ret:
            break
        k = cv.waitKey(1)
        
        # utile per impostare un frame rate di acquisizione manuale in secondi (to improve)
        time.sleep(time_sleep)

        # ESC to exit from webcam
        if k%256 == 27:
            print("Escape hit, closing...")
            break

        # fase di salvataggio frame
        else:
            # assegnazione del nome
            img_name = "{}{} ({}).jpg".format(directory, subject, img_counter)
            # salvataggio file
            cv.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            # conteggio per numerazione file
            img_counter += 1
            
            # check per limitare acquisizione
            if numbered:
                if counter == counter_limit:
                    print("Counter limit reached:{}".format(counter_limit))
                    break
                counter+=1

    # chiusura cam e windows cam
    cam.release()
    cv.destroyAllWindows()

- ### Happy person images :) 

In [48]:
from_directory = "images/fabrizio/raw_images/happy/"
to_directory = "images/fabrizio/happy/"

In [50]:
# Image acquisition

acquisition_cam(directory = from_directory, subject = "happy", init_number = 0,
                counter_limit = 110, time_sleep = 0.5, numbered = True)

images/fabrizio/raw_images/happy/happy (0).jpg written!
images/fabrizio/raw_images/happy/happy (1).jpg written!
images/fabrizio/raw_images/happy/happy (2).jpg written!
images/fabrizio/raw_images/happy/happy (3).jpg written!
images/fabrizio/raw_images/happy/happy (4).jpg written!
images/fabrizio/raw_images/happy/happy (5).jpg written!
images/fabrizio/raw_images/happy/happy (6).jpg written!
images/fabrizio/raw_images/happy/happy (7).jpg written!
images/fabrizio/raw_images/happy/happy (8).jpg written!
images/fabrizio/raw_images/happy/happy (9).jpg written!
images/fabrizio/raw_images/happy/happy (10).jpg written!
images/fabrizio/raw_images/happy/happy (11).jpg written!
images/fabrizio/raw_images/happy/happy (12).jpg written!
images/fabrizio/raw_images/happy/happy (13).jpg written!
images/fabrizio/raw_images/happy/happy (14).jpg written!
images/fabrizio/raw_images/happy/happy (15).jpg written!
images/fabrizio/raw_images/happy/happy (16).jpg written!
images/fabrizio/raw_images/happy/happy (1

In [51]:
# Image cropping

write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 79/110 images cropped


- ### Neutral person images :|

In [52]:
from_directory = "images/fabrizio/raw_images/neutral/"
to_directory = "images/fabrizio/neutral/"

In [53]:
acquisition_cam(directory = from_directory, subject = "neutral", init_number = 0,
                counter_limit = 110, time_sleep = 0.5, numbered = True)

images/fabrizio/raw_images/neutral/neutral (0).jpg written!
images/fabrizio/raw_images/neutral/neutral (1).jpg written!
images/fabrizio/raw_images/neutral/neutral (2).jpg written!
images/fabrizio/raw_images/neutral/neutral (3).jpg written!
images/fabrizio/raw_images/neutral/neutral (4).jpg written!
images/fabrizio/raw_images/neutral/neutral (5).jpg written!
images/fabrizio/raw_images/neutral/neutral (6).jpg written!
images/fabrizio/raw_images/neutral/neutral (7).jpg written!
images/fabrizio/raw_images/neutral/neutral (8).jpg written!
images/fabrizio/raw_images/neutral/neutral (9).jpg written!
images/fabrizio/raw_images/neutral/neutral (10).jpg written!
images/fabrizio/raw_images/neutral/neutral (11).jpg written!
images/fabrizio/raw_images/neutral/neutral (12).jpg written!
images/fabrizio/raw_images/neutral/neutral (13).jpg written!
images/fabrizio/raw_images/neutral/neutral (14).jpg written!
images/fabrizio/raw_images/neutral/neutral (15).jpg written!
images/fabrizio/raw_images/neutral

In [54]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 95/110 images cropped


- ### Sad person images :(

In [55]:
from_directory = "images/fabrizio/raw_images/sad/"
to_directory = "images/fabrizio/sad/"

In [56]:
acquisition_cam(directory = from_directory, subject = "sad", init_number = 0,
                counter_limit = 110, time_sleep = 0.5, numbered = True)

images/fabrizio/raw_images/sad/sad (0).jpg written!
images/fabrizio/raw_images/sad/sad (1).jpg written!
images/fabrizio/raw_images/sad/sad (2).jpg written!
images/fabrizio/raw_images/sad/sad (3).jpg written!
images/fabrizio/raw_images/sad/sad (4).jpg written!
images/fabrizio/raw_images/sad/sad (5).jpg written!
images/fabrizio/raw_images/sad/sad (6).jpg written!
images/fabrizio/raw_images/sad/sad (7).jpg written!
images/fabrizio/raw_images/sad/sad (8).jpg written!
images/fabrizio/raw_images/sad/sad (9).jpg written!
images/fabrizio/raw_images/sad/sad (10).jpg written!
images/fabrizio/raw_images/sad/sad (11).jpg written!
images/fabrizio/raw_images/sad/sad (12).jpg written!
images/fabrizio/raw_images/sad/sad (13).jpg written!
images/fabrizio/raw_images/sad/sad (14).jpg written!
images/fabrizio/raw_images/sad/sad (15).jpg written!
images/fabrizio/raw_images/sad/sad (16).jpg written!
images/fabrizio/raw_images/sad/sad (17).jpg written!
images/fabrizio/raw_images/sad/sad (18).jpg written!
ima

In [57]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 108/110 images cropped


- ### Angry person images è.é

In [58]:
from_directory = "images/fabrizio/raw_images/angry/"
to_directory = "images/fabrizio/angry/"

In [59]:
acquisition_cam(directory = from_directory, subject = "angry", init_number = 0,
                counter_limit = 110, time_sleep = 0.5, numbered = True)

images/fabrizio/raw_images/angry/angry (0).jpg written!
images/fabrizio/raw_images/angry/angry (1).jpg written!
images/fabrizio/raw_images/angry/angry (2).jpg written!
images/fabrizio/raw_images/angry/angry (3).jpg written!
images/fabrizio/raw_images/angry/angry (4).jpg written!
images/fabrizio/raw_images/angry/angry (5).jpg written!
images/fabrizio/raw_images/angry/angry (6).jpg written!
images/fabrizio/raw_images/angry/angry (7).jpg written!
images/fabrizio/raw_images/angry/angry (8).jpg written!
images/fabrizio/raw_images/angry/angry (9).jpg written!
images/fabrizio/raw_images/angry/angry (10).jpg written!
images/fabrizio/raw_images/angry/angry (11).jpg written!
images/fabrizio/raw_images/angry/angry (12).jpg written!
images/fabrizio/raw_images/angry/angry (13).jpg written!
images/fabrizio/raw_images/angry/angry (14).jpg written!
images/fabrizio/raw_images/angry/angry (15).jpg written!
images/fabrizio/raw_images/angry/angry (16).jpg written!
images/fabrizio/raw_images/angry/angry (1

In [60]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 103/110 images cropped


- ### Surprised person images :O

In [64]:
from_directory = "images/fabrizio/raw_images/surprise/"
to_directory = "images/fabrizio/surprise/"

In [65]:
acquisition_cam(directory = from_directory, subject = "surprise", init_number = 0,
                counter_limit = 110, time_sleep = 0.5, numbered = True)

images/fabrizio/raw_images/surprise/surprise (0).jpg written!
images/fabrizio/raw_images/surprise/surprise (1).jpg written!
images/fabrizio/raw_images/surprise/surprise (2).jpg written!
images/fabrizio/raw_images/surprise/surprise (3).jpg written!
images/fabrizio/raw_images/surprise/surprise (4).jpg written!
images/fabrizio/raw_images/surprise/surprise (5).jpg written!
images/fabrizio/raw_images/surprise/surprise (6).jpg written!
images/fabrizio/raw_images/surprise/surprise (7).jpg written!
images/fabrizio/raw_images/surprise/surprise (8).jpg written!
images/fabrizio/raw_images/surprise/surprise (9).jpg written!
images/fabrizio/raw_images/surprise/surprise (10).jpg written!
images/fabrizio/raw_images/surprise/surprise (11).jpg written!
images/fabrizio/raw_images/surprise/surprise (12).jpg written!
images/fabrizio/raw_images/surprise/surprise (13).jpg written!
images/fabrizio/raw_images/surprise/surprise (14).jpg written!
images/fabrizio/raw_images/surprise/surprise (15).jpg written!
im

In [66]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 13/110 images cropped


- ### Disgusted person images Dx

In [67]:
from_directory = "images/fabrizio/raw_images/disgust/"
to_directory = "images/fabrizio/disgust/"

In [68]:
acquisition_cam(directory = from_directory, subject = "disgust", init_number = 0,
                counter_limit = 110, time_sleep = 0.5, numbered = True)

images/fabrizio/raw_images/disgust/disgust (0).jpg written!
images/fabrizio/raw_images/disgust/disgust (1).jpg written!
images/fabrizio/raw_images/disgust/disgust (2).jpg written!
images/fabrizio/raw_images/disgust/disgust (3).jpg written!
images/fabrizio/raw_images/disgust/disgust (4).jpg written!
images/fabrizio/raw_images/disgust/disgust (5).jpg written!
images/fabrizio/raw_images/disgust/disgust (6).jpg written!
images/fabrizio/raw_images/disgust/disgust (7).jpg written!
images/fabrizio/raw_images/disgust/disgust (8).jpg written!
images/fabrizio/raw_images/disgust/disgust (9).jpg written!
images/fabrizio/raw_images/disgust/disgust (10).jpg written!
images/fabrizio/raw_images/disgust/disgust (11).jpg written!
images/fabrizio/raw_images/disgust/disgust (12).jpg written!
images/fabrizio/raw_images/disgust/disgust (13).jpg written!
images/fabrizio/raw_images/disgust/disgust (14).jpg written!
images/fabrizio/raw_images/disgust/disgust (15).jpg written!
images/fabrizio/raw_images/disgust

In [69]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 57/110 images cropped


- ### Fearful person images :S

In [74]:
from_directory = "images/fabrizio/raw_images/fear/"
to_directory = "images/fabrizio/fear/"

In [75]:
acquisition_cam(directory = from_directory, subject = "fear", init_number = 0,
                counter_limit = 110, time_sleep = 0.5, numbered = True)

images/fabrizio/raw_images/fear/fear (0).jpg written!
images/fabrizio/raw_images/fear/fear (1).jpg written!
images/fabrizio/raw_images/fear/fear (2).jpg written!
images/fabrizio/raw_images/fear/fear (3).jpg written!
images/fabrizio/raw_images/fear/fear (4).jpg written!
images/fabrizio/raw_images/fear/fear (5).jpg written!
images/fabrizio/raw_images/fear/fear (6).jpg written!
images/fabrizio/raw_images/fear/fear (7).jpg written!
images/fabrizio/raw_images/fear/fear (8).jpg written!
images/fabrizio/raw_images/fear/fear (9).jpg written!
images/fabrizio/raw_images/fear/fear (10).jpg written!
images/fabrizio/raw_images/fear/fear (11).jpg written!
images/fabrizio/raw_images/fear/fear (12).jpg written!
images/fabrizio/raw_images/fear/fear (13).jpg written!
images/fabrizio/raw_images/fear/fear (14).jpg written!
images/fabrizio/raw_images/fear/fear (15).jpg written!
images/fabrizio/raw_images/fear/fear (16).jpg written!
images/fabrizio/raw_images/fear/fear (17).jpg written!
images/fabrizio/raw_

In [76]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 109/110 images cropped


&nbsp;

## Cropping Google images

In [2]:
def write_imgs(from_directory, to_directory, init_number = 1,
              classifier = 'haarcascade_frontalface_default.xml', scale_factor = 1.1, minNeighbors = 5,
              size = (224, 224), color = "RGB"):
    '''
    This function takes in input images and after face detection apply a crop and a resize to that images.
    
    @params:
    from_directory = where to find the photos
    to_directory = where to save the photos
    init_number = first number to start naming photos
    classifier = face detector pre-trained (XML file)
    scale_factor = value for classifier (from (1,inf), low values increase false positives)
    minNeighbors = value for classifier (high values decrease false positives)
    size = tuple, format of the saved image
    color = RGB or BW (to save)
    
    '''
    # viene assegnato il classificatore
    face_cascade = cv.CascadeClassifier(classifier)
    
    # variabili per conteggio ed errori
    flag = init_number
    error = 0
    
    for filename in os.listdir(from_directory):
        # check per leggere solamente file jpg o jpeg
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            # lettura dell'immagine            
            img = cv.imread(os.path.join(from_directory, filename))
            # conversione immagine necessaria per il face detector
            gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
            # esecuzione face detector
            faces = face_cascade.detectMultiScale(gray, scale_factor, minNeighbors)
            # check necessario per assegnare solo immagini in cui vengono trovati volti (altrimenti doppioni a catena!)
            if len(faces) == 1:
                for (x, y, w, h) in faces:
                    # ritaglio dell'immagine rispetto al "quadrato" trovato
                    roi_color = img[y:y+h, x:x+w]
                    # resize dell'immagine
                    n_img = cv.resize(roi_color, size)
                    
                    # salvataggio immagine
                    if color == "RGB":
                        cv.imwrite(os.path.join(to_directory, str(flag))+"google_rgb.jpg", n_img)
                        flag += 1
                    elif color == "BW":
                        cv.imwrite(os.path.join(to_directory, str(flag))+"google_bw.jpg", cv.cvtColor(n_img, cv.COLOR_RGB2GRAY))
                        flag += 1
                        
            # se non trova volti, prosegue qui            
            else:
                error += 1
                try:
                    # delete immagine per non creare doppioni (more safe!)
                    del faces, n_img
                # necessaria se non ha trovato volti (evitare crash code)
                except:
                    pass
                
    # output finale           
    print('Created {}/{} images cropped'.format(flag-1, flag+error-1))
                    
    

- ### Happy Google :) 

In [22]:
from_directory = "images/google_dataset/raw_images/happy/"
to_directory = "images/google_dataset/happy/"

In [24]:
# Image cropping

write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 56/102 images cropped


- ### Neutral Google :|

In [7]:
from_directory = "images/google_dataset/raw_images/neutral/"
to_directory = "images/google_dataset/neutral/"

In [9]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 74/98 images cropped


- ### Sad Google :(

In [14]:
from_directory = "images/google_dataset/raw_images/sad/"
to_directory = "images/google_dataset/sad/"

In [15]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 64/122 images cropped


- ### Angry Google è.é

In [12]:
from_directory = "images/google_dataset/raw_images/angry/"
to_directory = "images/google_dataset/angry/"

In [13]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 88/148 images cropped


- ### Surprised Google :O

In [3]:
from_directory = "images/google_dataset/raw_images/surprise/"
to_directory = "images/google_dataset/surprise/"

In [ ]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

- ### Disgusted Google Dx

In [18]:
from_directory = "images/google_dataset/raw_images/disgust/"
to_directory = "images/google_dataset/disgust/"

In [19]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 68/100 images cropped


- ### Fearful Google :S

In [20]:
from_directory = "images/google_dataset/raw_images/fear/"
to_directory = "images/google_dataset/fear/"

In [21]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 44/91 images cropped


&nbsp;

## Cropping test images

In [2]:
def write_imgs(from_directory, to_directory, init_number = 1,
              classifier = 'haarcascade_frontalface_default.xml', scale_factor = 1.1, minNeighbors = 5,
              size = (224, 224), color = "RGB"):
    '''
    This function takes in input images and after face detection apply a crop and a resize to that images.
    
    @params:
    from_directory = where to find the photos
    to_directory = where to save the photos
    init_number = first number to start naming photos
    classifier = face detector pre-trained (XML file)
    scale_factor = value for classifier (from (1,inf), low values increase false positives)
    minNeighbors = value for classifier (high values decrease false positives)
    size = tuple, format of the saved image
    color = RGB or BW (to save)
    
    '''
    # viene assegnato il classificatore
    face_cascade = cv.CascadeClassifier(classifier)
    
    # variabili per conteggio ed errori
    flag = init_number
    error = 0
    
    for filename in os.listdir(from_directory):
        # check per leggere solamente file jpg o jpeg
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".JPG"):
            # lettura dell'immagine            
            img = cv.imread(os.path.join(from_directory, filename))
            # conversione immagine necessaria per il face detector
            gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
            # esecuzione face detector
            faces = face_cascade.detectMultiScale(gray, scale_factor, minNeighbors)
            # check necessario per assegnare solo immagini in cui vengono trovati volti (altrimenti doppioni a catena!)
            if len(faces) == 1:
                for (x, y, w, h) in faces:
                    # ritaglio dell'immagine rispetto al "quadrato" trovato
                    roi_color = img[y:y+h, x:x+w]
                    # resize dell'immagine
                    n_img = cv.resize(roi_color, size)
                    
                    # salvataggio immagine
                    if color == "RGB":
                        cv.imwrite(os.path.join(to_directory, str(flag))+"test_rgb.jpg", n_img)
                        flag += 1
                    elif color == "BW":
                        cv.imwrite(os.path.join(to_directory, str(flag))+"test_bw.jpg", cv.cvtColor(n_img, cv.COLOR_RGB2GRAY))
                        flag += 1
                        
            # se non trova volti, prosegue qui            
            else:
                error += 1
                try:
                    # delete immagine per non creare doppioni (more safe!)
                    del faces, n_img
                # necessaria se non ha trovato volti (evitare crash code)
                except:
                    pass
                
    # output finale           
    print('Created {}/{} images cropped'.format(flag-1, flag+error-1))
                    
    

- ### Test :) 

In [22]:
from_directory = "images/test/raw_images/happy/"
to_directory = "images/test/happy/"

In [23]:
# Image cropping

write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 15/15 images cropped


- ### Test :|

In [25]:
from_directory = "images/test/raw_images/neutral/"
to_directory = "images/test/neutral/"

In [26]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 15/15 images cropped


- ### Test :(

In [33]:
from_directory = "images/test/raw_images/sad/"
to_directory = "images/test/sad/"

In [34]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.1, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 15/15 images cropped


- ### Test è.é

In [29]:
from_directory = "images/test/raw_images/angry/"
to_directory = "images/test/angry/"

In [30]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 15/15 images cropped


- ### Test :O

In [7]:
from_directory = "images/test/raw_images/surprise/"
to_directory = "images/test/surprise/"

In [8]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.01, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 11)

Created 1/1 images cropped


- ### Test Dx

In [40]:
from_directory = "images/test/raw_images/disgust/"
to_directory = "images/test/disgust/"

In [42]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.1, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 15/15 images cropped


- ### Test :S

In [45]:
from_directory = "images/test/raw_images/fear/"
to_directory = "images/test/fear/"

In [46]:
write_imgs(from_directory = from_directory, to_directory = to_directory, color = "RGB",
           scale_factor = 1.2, classifier = 'haarCascade/haarcascade_frontalface_alt2.xml', minNeighbors = 8)

Created 15/15 images cropped


&nbsp;